## Muestreo

Para analizar la calidad del conjunto de datos, seleccionar variables, limpiar y transformar los datos y finalmente determinar un número $k$ de clusters partiremos de un muestreo del conjunto de datos inicial, con un tamaño de muestra del 20% con respecto al original (porque la memoria RAM nos lo permite). 

Para el muestreo utilizamos el método de __reservoir sapling__ visto en clase.

Como el conjunto de datos cuenta con más de 5 millones de registros, segun su [documentación](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3) fijamos el tamaño de muestra:




In [ ]:
n_muestras <- 5.4*(1*10**6)/5
t1 <- Sys.time()
dir()
print(n_muestras)
set.seed(0)

In [ ]:
path.file <- "Fire_Department_Calls_for_Service.csv" #--path donde se encuentra el conjunto de datos original 
n_filas_read <- 500000 # la rm nos permite cargar este numero de registros y agilizar el muestreo

connection <- file(path.file, open = "r")
#--nombre de las columnas (primera fila)
#col_names  <- read.csv(connection, nrows = 1, header = TRUE)
#--definimos nuestro buffer (muestra) y los rellenamos con las primeras n_muestras filas
buffer <- read.csv(connection, nrows = n_muestras, header = TRUE, stringsAsFactors = FALSE)

#--indice que nos permitira generar los numeros aleatorios correctamente
posicion_inicial <- n_muestras
random_unif <- function(x) sample.int(x,1)

contador <- 1
repeat{
  print(paste0("Posicion inicial: ", posicion_inicial))
  
   #--leemos una parte del archivo
  temp <- read.csv(connection, nrows = n_filas_read, header = FALSE)

  #--indices que controlan el maximo de cada numero aleatorio
  maximo <- c(1:nrow(temp)) + posicion_inicial

  #--generamos numeros aleatorios de forma vectorizada, segun esto solo permuta los indices
  j = vapply(maximo, random_unif, FUN.VALUE = integer(1))

  #--observamos cuales de los numeros aleatorios son menores que nuestra muestra
  idx <- j <= n_muestras

  #--sustituimos los que resultaron menores
  buffer[j[idx], ] <- temp[idx, ]
  
  print(paste0("iteracion: ", contador))
  contador <- contador + 1
  #--redefinimos la posicion inicial para la siguiente iteracion
  posicion_inicial <- posicion_inicial + nrow(temp)
  
  #--si el numero de filas leidas es menor que el esperado,
  #-asumimos que se acabo el archivo y salimos del ciclo
  if(nrow(temp) < n_filas_read)
    break
}
t2 <- Sys.time()
print(t2 - t1) #(Time difference of 9.29019 mins mins )
#--guardar nuestra muestra para un futuro analisis
write.csv(buffer, paste0("muestra_", path.file), row.names = FALSE )
close(connection)

In [ ]:
library(readr)
data <- read_csv('muestra3_Fire_Department_Calls_for_Service.csv')

In [ ]:
print(dim(data))

Después de leer la documentación y entender la estructura de la data, decidimos que el cluster que realizaremos tendrá como objetivo encontrar grupos de llamadas parecidas entre sí y contrastaremos estos grupos con la etiqueta que poseen los datos en la columna `Call Type Group`.
Después de revisar la documentación  descartamos las columnas `Call.Type`, `RowID`, `Unit ID`, `Incident Number`, `Unit Type`, `Unit sequence in call dispatch` al igual que `Location`. Las primeras por no aportar información extra y la última porque la documentación no proporciona el tipo de proyección utilizado para referencias las coordenadas de los puntos.

In [ ]:
data$RowID <- data$Unit.ID <- data$Incident.Number <- data$Location <- data$Unit.Type <- data$Unit.sequence.in.call.dispatch <- data$Call.Type <- NULL

In [ ]:
names(data)

En vista de que las siguientes columnas no se encuentran [documentadas en el diccionario de datos correspondiente](https://data.sfgov.org/api/views/nuek-vuh3/files/ddb7f3a9-0160-4f07-bb1e-2af744909294?download=true&filename=FIR-0002_DataDictionary_fire-calls-for-service.xlsx) procedemos a eliminarlas:
`Current Police Districts`, `Neighborhoods - Analysis Boundaries` , `Zip Codes`, `Neighborhoods (old)`, `Police Districts`, `Civic Center Harm Reduction Project Boundary`, `HSOC Zones` y  `Central Market/Tenderloin Boundary Polygon - Updated`

In [ ]:
data[ , c('Current.Police.Districts', 'Neighborhoods...Analysis.Boundaries', 'Zip.Codes', 'Neighborhoods..old.', 'Police.Districts', 
          'Civic.Center.Harm.Reduction.Project.Boundary', 'HSOC.Zones', 'Central.Market.Tenderloin.Boundary.Polygon...Updated')] <- NULL

In [ ]:
head(data)

Procedemos a retener el último registro de cada llamada, el cual contiene la información acumulada de los anteriores. 

In [ ]:
library(dplyr)
library(lubridate)
data$Received.DtTm <- mdy_hms(data$Received.DtTm)
data %>% group_by(Call.Number ) %>% arrange(Call.Number , Received.DtTm ) %>% mutate( flag1 = n() ,flag2 = row_number()) -> data
data %>% filter(flag1 ==flag2 ) -> data
data$flag1 <- data$flag2 <- NULL
data$Available.DtTm <- NULL
data$Call.Date <- mdy(data$Call.Date)
data$Watch.Date <- mdy(data$Watch.Date)
data$Entry.DtTm <- mdy_hms(data$Entry.DtTm)
data$Dispatch.DtTm <- mdy_hms(data$Dispatch.DtTm)
data$Response.DtTm <- mdy_hms( data$Response.DtTm) 
data$On.Scene.DtTm <- mdy_hms( data$On.Scene.DtTm) 
data$Transport.DtTm <- mdy_hms(data$Transport.DtTm) 
data$Hospital.DtTm <- mdy_hms(data$Hospital.DtTm) 


Como suponemos que la duración de la llamada está correlacionada con su clasificación con las variables de tipo fecha (`Call.Date`, `Watch.Date`, `Received.DtTm`, `Entry.DtTm`, `Dispatch.DtTm`, `Response.DtTm`, `On.Scene.DtTm`, `Transport.DtTm`, `Hospital.DtTm`) obtenemos la duración aproximada de la llamada. 


In [ ]:
head(data)

In [ ]:
#install.packages('reshape2')
library(reshape2)

In [ ]:
data.t <- melt(data, id = c('Call.Number','Call.Final.Disposition', 'Address', 'City', 'Zipcode.of.Incident', 'Battalion', 
                            'Station.Area', 'Box', 'Original.Priority', 'Priority', 'Final.Priority', 'ALS.Unit', 'Call.Type.Group', 
                            'Number.of.Alarms',  'Fire.Prevention.District', 'Supervisor.District', 
                            'Neighborhooods...Analysis.Boundaries', 'Supervisor.Districts', 'Fire.Prevention.Districts' ) ) %>%
          filter( variable %in% c('Received.DtTm', 'Entry.DtTm', 'Dispatch.DtTm', 'Response.DtTm', 'On.Scene.DtTm', 'Transport.DtTm', 'Hospital.DtTm' )) 
head(data.t)

In [ ]:
data.t %>% group_by (Call.Number, Call.Final.Disposition, Address, City, Zipcode.of.Incident, Battalion, 
                            Station.Area, Box, Original.Priority, Priority, Final.Priority, ALS.Unit, Call.Type.Group, 
                            Number.of.Alarms,  Fire.Prevention.District, Supervisor.District, 
                            Neighborhooods...Analysis.Boundaries, Supervisor.Districts, Fire.Prevention.Districts  ) %>% 
      summarise( min.t = min(value, na.rm=TRUE), max.t =max(value, na.rm=TRUE)) -> data.t
data.t <- data.t %>% mutate(Call.seconds = max.t - min.t)
data.t$min.t <- data.t$max.t <- NULL 

In [ ]:
head(data.t)

In [ ]:
table(data.t$Call.Type.Group)

## K-means on-line

In [ ]:
alpha <- 0.1
kmeans.online.b.init <- function(data, k, alpha){
  # clousure para distribuir la eleccion del elemento k
  data <- data
  alpha <- alpha
  function(k){
    # Entradas 
    # data (data.frame): Dataframe donde las observaciones son los elementos a clusterizar y las columnas son las variables
    # k (int): Numero de cluster requerido
    # alpha (numeric): learning rate
    # Salida
    # kmeans.online con los elementos:
    # tabla.master (data.frame): Dos columnas, la primera con el id de la observacion y la segunda con el label del cluster
    # statas.intra (vector): Vector con la media d ela varianza intraelementos
    tabla.master <- data.frame(Obs = row.names(data), Cluster= rep(-Inf, dim(data)[1]))
    # inicializacion alatoria entre el minimo y maximo de cada variable
    stats.min <- sapply(data, min)
    stats.max <- sapply(data, max)
    set.seed(0)
    centroides <- mapply(function(x, y) {runif(k, x, y)},  stats.min, stats.max) 
    # termina inicializacion de centroides
    
    # comienza kmeans proceso online
    
    for( i in 1:dim(data)[1])
    {
      #i <- 11
      #print(i)
      # comienza asignacion de cluster mas cercano
      observacion.en.juego <- as.matrix(data[i, ])
      m.temp <- as.matrix(rbind(observacion.en.juego, centroides))
      distancias <- dist(m.temp)
      m.distancias <- as.matrix(distancias)
      k.i <- which.min(m.distancias[1, 2:(k+1)])
      tabla.master$Cluster[i] <- k.i
      # termina asignacion de cluster m�s cercano 
      # update de cluster
      centroides[k.i, ] <- centroides[k.i, ] + alpha*observacion.en.juego
    }
    stats <- rep(-Inf, k)
    for ( i in 1:k)
    {
      index <- which( tabla.master$Cluster == i)
      data.subset <- docs.vector[ index, ]
      stats.i <- dist(data.subset)
      stats[i] <- sum(stats.i) # asumimos independencia entre las variables
    }
    kmeans.online <- list( tabla.master =tabla.master, statas.intra = stats)
    return(kmeans.online)
  }
}
data <- as.data.frame(data.t)

In [ ]:
Y <- data.frame(y=data$Call.Type.Group)
row.names(Y) <- row.names(data) <- data$Call.Number
data$Call.Type.Group <- data$Call.Number <- NULL
# hacemos un cambio de encoding 

In [ ]:
index <- which(sapply(data, class) == 'character')
normalizar <- function(x, na.rm = FALSE) (x - mean(x, na.rm = na.rm)) / sd(x, na.rm)

In [ ]:
for (i in index)
    {
    index.na <- which( is.na(data[, i] ))
    data[index.na, i] <- 'NULL'
    temp <- factor( data[, i])
    data[, i] <- normalizar(as.numeric( temp))
}

In [ ]:
head(data)
print(dim(data))

In [ ]:
kmeans.online.b <- kmeans.online.b.init(data = data, alpha = alpha)

In [ ]:
set.seed(0)
cluster <- 1:3
for( i in 1:3)
{
  print(i)
  res <- kmeans.online.b(k=i+1)
  cluster[i] <- sum(res$statas.intra)
  print(cluster)
}

In [ ]:
1

In [ ]:
plot((cluster),type = 'l', xlab = 'Multiplos de 5', ylab = 'Var intra cluster')
plot(abs(diff(cluster)),type = 'l',  xlab = 'Multiplos de 5', ylab = 'diff(Var) intra cluster')
